<a href="https://colab.research.google.com/github/PeptoneLtd/nerfax/blob/main/notebooks/Reconstruct_Protein_Backbone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/PeptoneLtd/nerfax.git
import mdtraj as md
import numpy as np
import nerfax
from nerfax.parser import xyz_to_internal_coords, insert_zero
from nerfax.foldcomp_tests import compute_rmsd

path = 'model.pdb'
!wget https://alphafold.ebi.ac.uk/files/AF-Q93WI9-F1-model_v4.pdb -q -O {path}
t = md.load(path)
t = t.atom_slice(t.top.select('resid < 50'))

xyz_bb = t.xyz[0,t.top.select('name CA or name C or name N')]*10 # xyz_bb in Angstrom

# Pull out lengths, angles and dihedrals
lengths, angles, dihedrals = xyz_to_internal_coords(insert_zero(xyz_bb.reshape(-1,3,3)).reshape(-1,3)) # Dummys required for the first residue as it has a set of 'ghost' dihedrals and angles
angles = angles.at[0,0].set(1.) # (Dummy non-zero angle required)

xyz_bb_reconstructed = nerfax.reconstruct.reconstruct_from_internal_coordinates(lengths, angles, dihedrals, mode='fully_sequential')

# We can realign this back to the reference frame, and get an RMSD
xyz_bb_reconstructed_aligned = nerfax.utils.get_align_rigid_bodies_fn(xyz_bb_reconstructed, xyz_bb)(xyz_bb_reconstructed)
print(f'RMSD (with explicit angles and lengths): {compute_rmsd(xyz_bb_reconstructed_aligned, xyz_bb):.1e} Angstrom')


## If using just the dihedrals, we can used idealised angles and bond lengths
BACKBONE_BOND_LENGTHS = np.array([1.33,1.46,1.52])
lengths_idealised = np.broadcast_to(BACKBONE_BOND_LENGTHS[None], dihedrals.shape)
BACKBONE_ANGLES = np.array([1.095, 0.98, 1.20])
angles_idealised = np.broadcast_to(BACKBONE_ANGLES[None], dihedrals.shape) # Due to scnet's definition of the angle, this is actually 180 - angle (so ~60 degrees rather 120)

xyz_bb_reconstructed = nerfax.reconstruct.reconstruct_from_internal_coordinates(lengths_idealised, angles_idealised, dihedrals, mode='fully_sequential')

# We can realign this back to the reference frame, and get an RMSD
xyz_bb_reconstructed_aligned = nerfax.utils.get_align_rigid_bodies_fn(xyz_bb_reconstructed, xyz_bb)(xyz_bb_reconstructed)
print(f'RMSD (with idealised angles and lengths): {compute_rmsd(xyz_bb_reconstructed_aligned, xyz_bb):.1e} Angstrom')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/mdtraj/formats/pdb/pdbfile.py:200: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


RMSD (with explicit angles and lengths): 1.6e-05 Angstrom
RMSD (with idealised angles and lengths): 7.4e+00 Angstrom


In [2]:
!pip install -q nglview
from google.colab import output
output.enable_custom_widget_manager()
import nglview as nv
from copy import deepcopy
t_reconstruct = deepcopy(t.atom_slice(t.top.select('name CA or name C or name N')))
t_reconstruct.xyz = xyz_bb_reconstructed_aligned[None]/10.

vw = nv.NGLWidget()
vw.add_structure(nv.MDTrajTrajectory(t))
vw.add_structure(nv.MDTrajTrajectory(t_reconstruct))
vw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00


NGLWidget()